In [2]:
import pandas as pd


# emails = pd.read_csv("emails.csv")

# emails.columns = ['file', 'message']
# pd.set_option('display.max_colwidth', None)

# id_email = pd.read_csv("id-email.csv")
# sender = id_email.iloc[120][1]
# receiver = id_email.iloc[2799][1]
# print(sender)
# print(receiver)


# matches = emails[
#     # emails['message'].str.contains("2 May 2001") &
#     emails['message'].str.contains(sender) &
#     emails['message'].str.contains(receiver)
# ]

# print(matches.iloc[0]['message'])

base_date = pd.Timestamp('1999-01-01', tz='UTC')

def day_index_to_date(day_index: int, tz: str = 'UTC') -> pd.Timestamp:
    di = int(day_index)
    dt = base_date + pd.Timedelta(days=di)
    # Convert timezone if requested
    return dt.tz_convert(tz) if tz and dt.tzinfo is not None and tz != 'UTC' else dt

def format_human_date(ts) -> str:
    """Return dates like '4 May 2001' (no leading zero), Windows-safe."""
    ts = pd.Timestamp(ts)
    for fmt in ('%-d %B %Y', '%#d %B %Y', '%d %B %Y'):
        try:
            s = ts.strftime(fmt)
            if fmt == '%d %B %Y':
                s = s.lstrip('0')
            return s
        except ValueError:
            continue
    return ts.strftime('%Y-%m-%d')


day = day_index_to_date(410)
string_day = " "+ format_human_date(day) #add space to the beginning of the string because the first character is a space and if not putten may cause errors like .contains("4 May 2001") will back you 14 May 2001
string_day





' 15 February 2000'

In [3]:
emails = pd.read_csv("emails.csv")
modified = pd.read_csv("modified-emails.csv")

# Append rows (keeps columns "file" and "message")
merged = pd.concat([emails, modified], ignore_index=True)

# Save result
merged.to_csv("emails_synthetic.csv", index=False)

print("Merged CSV saved as emails_merged.csv")

Merged CSV saved as emails_merged.csv


In [5]:
import csv
import ast

# Define the new attacker nodes
new_attackers = {
    6600: {200: 1, 201: 0, 202: 1, 203: 0, 205: 1, 206: 0, 208: 1, 209: 0, 210: 2, 211: 0},
    6601: {215: 1, 216: 0, 217: 1, 218: 0, 220: 1, 221: 0, 223: 1, 224: 0, 225: 2, 226: 0},
    6602: {300: 6, 301: 3, 302: 3, 304: 1},
    6603: {50: 1, 100: 2, 120: 1, 190: 1, 210: 1, 280: 2, 290: 1, 360: 2, 380: 0, 410: 0}
}

# Define responses from target nodes (victim replies)
target_responses = {
    97: {201: 1, 203: 1, 206: 1, 209: 1, 211: 1},  # Campaign 1
    171: {216: 1, 218: 1, 221: 1, 224: 1, 226: 1},  # Campaign 2
    221: {301: 3, 302: 1},  # Campaign 3
    223: {301: 1},  # Campaign 3
    226: {301: 1},  # Campaign 3
    227: {302: 1},  # Campaign 3
    817: {380: 1},  # Campaign 4
    1017: {410: 1}  # Campaign 4
}

def merge_day_counts(existing_dict, new_dict):
    """Merge two dictionaries, adding values for matching keys"""
    result = existing_dict.copy()
    for day, count in new_dict.items():
        if day in result:
            result[day] += count
        else:
            result[day] = count
    return result

# Read the existing nodes.csv
nodes_data = {}
with open('nodes.csv', 'r') as f:
    reader = csv.reader(f)
    for row in reader:
        # Skip empty rows
        if not row or len(row) < 2:
            continue
        try:
            node_id = int(row[0])
            day_counts = ast.literal_eval(row[1])
            nodes_data[node_id] = day_counts
        except (ValueError, SyntaxError) as e:
            print(f"Skipping invalid row: {row} - Error: {e}")
            continue

# Update existing target nodes with their responses
for node_id, responses in target_responses.items():
    if node_id in nodes_data:
        nodes_data[node_id] = merge_day_counts(nodes_data[node_id], responses)
    else:
        # If target node doesn't exist, create it with just the responses
        nodes_data[node_id] = responses

# Add new attacker nodes
for node_id, day_counts in new_attackers.items():
    nodes_data[node_id] = day_counts

# Write to nodes-synthetic.csv
with open('nodes-synthetic.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    for node_id in sorted(nodes_data.keys()):
        writer.writerow([node_id, str(nodes_data[node_id])])

print("Successfully created nodes-synthetic.csv!")
print(f"Total nodes: {len(nodes_data)}")
print(f"New attacker nodes added: {list(new_attackers.keys())}")
print(f"Target nodes updated: {list(target_responses.keys())}")

Successfully created nodes-synthetic.csv!
Total nodes: 6604
New attacker nodes added: [6600, 6601, 6602, 6603]
Target nodes updated: [97, 171, 221, 223, 226, 227, 817, 1017]


In [6]:
import csv

# Define synthetic attack edges based on campaigns
# Format: (From, To, Count)
synthetic_edges = [
    # Campaign 1: Attacker 6600 -> Target 97 (11 emails total)
    (6600, 97, 6),  # Attacker sends 6 emails
    (97, 6600, 5),  # Target replies 5 times
    
    # Campaign 2: Attacker 6601 -> Target 171 (11 emails total)
    (6601, 171, 6),  # Attacker sends 6 emails
    (171, 6601, 5),  # Target replies 5 times
    
    # Campaign 3: Attacker 6602 -> Multiple Targets (18 emails total)
    (6602, 221, 4),  # 4 emails to node 221
    (221, 6602, 4),  # 4 replies from node 221
    (6602, 223, 2),  # 2 emails to node 223
    (223, 6602, 1),  # 1 reply from node 223
    (6602, 225, 2),  # 2 emails to node 225
    (6602, 226, 2),  # 2 emails to node 226
    (226, 6602, 1),  # 1 reply from node 226
    (6602, 227, 2),  # 2 emails to node 227
    (227, 6602, 1),  # 1 reply from node 227
    
    # Campaign 4: Attacker 6603 -> Targets 817, 1017 (13 emails total)
    (6603, 817, 6),  # 6 emails to node 817
    (817, 6603, 1),  # 1 reply from node 817
    (6603, 1017, 6), # 6 emails to node 1017
    (1017, 6603, 1)  # 1 reply from node 1017
]

# Read existing edges
edges_data = []
with open('edges.csv', 'r') as f:
    reader = csv.reader(f)
    header = next(reader)  # Save header
    for row in reader:
        if not row or len(row) < 3:
            continue
        try:
            from_node = int(row[0])
            to_node = int(row[1])
            count = int(row[2])
            edges_data.append((from_node, to_node, count))
        except ValueError as e:
            print(f"Skipping invalid row: {row} - Error: {e}")
            continue

# Add synthetic edges
edges_data.extend(synthetic_edges)

# Sort by From node, then To node
edges_data.sort(key=lambda x: (x[0], x[1]))

# Write to edges-synthetic.csv
with open('edges-synthetic.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['From', 'To', 'Count'])  # Write header
    for edge in edges_data:
        writer.writerow(edge)

print("Successfully created edges-synthetic.csv!")
print(f"Total edges: {len(edges_data)}")
print(f"Synthetic edges added: {len(synthetic_edges)}")
print("\nSynthetic attack connections:")
print("Campaign 1 (6600 <-> 97): 11 emails")
print("Campaign 2 (6601 <-> 171): 11 emails")
print("Campaign 3 (6602 <-> 221,223,225,226,227): 18 emails")
print("Campaign 4 (6603 <-> 817,1017): 13 emails")

Successfully created edges-synthetic.csv!
Total edges: 50914
Synthetic edges added: 17

Synthetic attack connections:
Campaign 1 (6600 <-> 97): 11 emails
Campaign 2 (6601 <-> 171): 11 emails
Campaign 3 (6602 <-> 221,223,225,226,227): 18 emails
Campaign 4 (6603 <-> 817,1017): 13 emails


In [7]:
import csv

# Define new attacker email addresses
new_emails = {
    6600: "security.alert@enron-support.com",
    6601: "hr.dept@enron-internal.com",
    6602: "hr.updates@enron-internal.com",
    6603: "support.survey@enron-feedback.com"
}

# Read existing id-email.csv
id_email_data = {}
with open('id-email.csv', 'r') as f:
    reader = csv.reader(f)
    for row in reader:
        if not row or len(row) < 2:
            continue
        try:
            node_id = int(row[0])
            email = row[1]
            id_email_data[node_id] = email
        except ValueError as e:
            print(f"Skipping invalid row: {row} - Error: {e}")
            continue

# Add new attacker emails
for node_id, email in new_emails.items():
    id_email_data[node_id] = email

# Write to id-email-synthetic.csv
with open('id-email-synthetic.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    for node_id in sorted(id_email_data.keys()):
        writer.writerow([node_id, id_email_data[node_id]])

print("Successfully created id-email-synthetic.csv!")
print(f"Total email mappings: {len(id_email_data)}")
print(f"\nNew attacker emails added:")
for node_id, email in sorted(new_emails.items()):
    print(f"  {node_id}: {email}")

Successfully created id-email-synthetic.csv!
Total email mappings: 6604

New attacker emails added:
  6600: security.alert@enron-support.com
  6601: hr.dept@enron-internal.com
  6602: hr.updates@enron-internal.com
  6603: support.survey@enron-feedback.com
